# Lesson 1: Simple ReAct Agent from Scratch

- LLM thinks then decides what is the action to take, and an observation is returned, then repeats

In [53]:
import openai
import google.generativeai as genai

import re # regular expression
# import httpx # http client library, similar to requests but with async
import os

from dotenv import load_dotenv
load_dotenv()  # Load environment variables from .env file

from openai import OpenAI

In [7]:
client = OpenAI()

chat_completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

chat_completion.choices[0].message.content


In [54]:
# Google Gemini
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("The opposite of hot is")
print(response.text)

cold



In [10]:
## Defining an Agent Class from scratch
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
                        model="gpt-4o-mini", 
                        temperature=0,
                        messages=self.messages)
        return completion.choices[0].message.content
    

In [55]:
class GeminiAgent:
    def __init__(self, system=""):
        # Configure the API key
        genai.configure(api_key=os.environ["GEMINI_API_KEY"])
        
        # Create the model
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        
        # Store system message and initialize messages
        self.system = system
        self.messages = []
        
        # Add system message if provided
        if self.system:
            self.messages.append({
                'role': 'user', 
                'parts': [self.system]
            })

    def __call__(self, message):
        # Add user message
        self.messages.append({
            'role': 'user', 
            'parts': [message]
        })
        
        # Execute and get result
        result = self.execute()
        
        # Add assistant response to messages
        self.messages.append({
            'role': 'model', 
            'parts': [result]
        })
        
        return result

    def execute(self):
        # Generate content with temperature set to 0
        generation_config = genai.types.GenerationConfig(
            temperature=0.0
        )
        
        # Call Gemini model
        response = self.model.generate_content(
            self.messages,
            generation_config=generation_config
        )
        
        # Return the text of the response
        return response.text

In [11]:
# Implementing a ReAct Framework
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

In [12]:
# Define some available functions
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier": 
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

In [13]:
abot = Agent(prompt)

In [14]:
result = abot("How much does a toy poodle weigh?")
print(result)

Thought: I need to find the average weight of a Toy Poodle using the average_dog_weight action. 
Action: average_dog_weight: Toy Poodle
PAUSE


In [18]:
result = average_dog_weight("Toy Poodle")
result

'a toy poodles average weight is 7 lbs'

In [19]:
next_prompt = "Observation: {}".format(result)
abot(next_prompt)

"Answer: A Toy Poodle's average weight is 7 lbs."

In [20]:
# For debugging
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [24]:
# Re-initialize the abot agent with the original prompt
abot = Agent(prompt)

In [25]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
abot(question)

"Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll start by looking up the weight of each breed. \n\nAction: average_dog_weight: Border Collie\nPAUSE"

In [26]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)

Observation: a Border Collies average weight is 37 lbs


In [27]:
abot(next_prompt)

'Thought: Now that I have the weight of the Border Collie, I need to find the average weight of the Scottish Terrier to complete the calculation for their combined weight.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [28]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)

Observation: Scottish Terriers average 20 lbs


In [49]:
abot(next_prompt)

'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

In [30]:
next_prompt = "Observation: {}".format(eval("37 + 20"))
print(next_prompt)

Observation: 57


# Put it into a smarter loop
- the example above is too manual, but it shows what are the steps taken

In [48]:
action_re = re.compile('^Action: (\w+): (.*)$')   # python regular expression to selection action

In [46]:
# OpenAI Model Query
def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [50]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
query(question)

Thought: I need to find the average weights of both a Border Collie and a Scottish Terrier to calculate their combined weight. I'll start by looking up the weight of the Border Collie.  
Action: average_dog_weight: Border Collie  
PAUSE
 -- running average_dog_weight Border Collie  
Observation: An average dog weights 50 lbs
Thought: I have the weight for the Border Collie, which is 50 lbs. Now, I need to find the average weight of a Scottish Terrier to complete the calculation for their combined weight.  
Action: average_dog_weight: Scottish Terrier  
PAUSE
 -- running average_dog_weight Scottish Terrier  
Observation: An average dog weights 50 lbs
Thought: I have the weights for both dogs now. The Border Collie weighs 50 lbs and the Scottish Terrier also weighs 50 lbs. I can now calculate their combined weight.  
Action: calculate: 50 + 50  
PAUSE
 -- running calculate 50 + 50  
Observation: 100
Answer: The combined weight of a Border Collie and a Scottish Terrier is 100 lbs.


# Google Gemini

In [56]:
def gemini_query(question, max_turns=5):
    i = 0
    bot = GeminiAgent(prompt)
    next_prompt = question
    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)
        actions = [
            action_re.match(a) 
            for a in result.split('\n') 
            if action_re.match(a)
        ]
        if actions:
            # There is an action to run
            action, action_input = actions[0].groups()
            if action not in known_actions:
                raise Exception("Unknown action: {}: {}".format(action, action_input))
            print(" -- running {} {}".format(action, action_input))
            observation = known_actions[action](action_input)
            print("Observation:", observation)
            next_prompt = "Observation: {}".format(observation)
        else:
            return

In [57]:
question = """I have 2 dogs, a border collie and a scottish terrier. \
What is their combined weight"""
gemini_query(question)

Thought: I need to find the average weight of a Border Collie and a Scottish Terrier, then add those weights together.  I'll use the `average_dog_weight` action twice.

Action: average_dog_weight: Border Collie
PAUSE

 -- running average_dog_weight Border Collie
Observation: a Border Collies average weight is 37 lbs
Thought: I have the weight of the Border Collie. Now I need the weight of the Scottish Terrier.

Action: average_dog_weight: Scottish Terrier
PAUSE

 -- running average_dog_weight Scottish Terrier
Observation: Scottish Terriers average 20 lbs
Thought: I have both weights. Now I need to add them together.

Action: calculate: 37 + 20
PAUSE

 -- running calculate 37 + 20
Observation: 57
Answer: The combined weight of the two dogs is 57 lbs.

